In [2]:
import cv2 as cv

import matplotlib.pyplot as plt
import numpy as np

%matplotlib

Using matplotlib backend: MacOSX


In [14]:
cap = cv.VideoCapture(0)

In [15]:
_, image = cap.read()

#Seleccionando interactivamente los parametros de la ROI
(x, y, w, h) = cv.selectROI('Traking', image, False)

# Obteniendo la ROI
roi = image[y:y+h , x:x+w]


# Mostrar el recorte
cv.imshow("Region de interes", roi)
cv.waitKey(0)
cv.destroyAllWindows()

In [16]:
# Pasamos la ROI a HSV (los valores de H son más estables a cambios de intensidad)
hsv_roi =  cv.cvtColor(roi, cv.COLOR_BGR2HSV)
# Definimos una máscara para el histograma (Hue: 0..180, Saturation:60..255, Value:32..255)
mask = cv.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
# Y calculamos el histograma sobre esa máscara (toma solo el Hue: tonalidad)
roi_hist = cv.calcHist([hsv_roi],[0],mask,[180],[0,180])
# Normalizamos el histograma (entre 0 y 255 según indica el flag cv.NORM_MINMAX)
cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)

# Mostramos lo que se fue armando en relación a la ROI
#-----------------------------------------------------
plt.figure()
plt.imshow(mask)

# Muestro la imagen como para ver dónde definimos la ROI
#-------------------------------------------------------
plt.figure()
plt.imshow(image)

## No me fue muy bien con esto

In [17]:
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )

while(1):
    timer = cv.getTickCount()
    
    ret, frame = cap.read()
    
    if ret == True:
    
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
        dst = cv.calcBackProject([hsv],[0],roi_hist,[0,180],1)
    
        ret, track_window = cv.meanShift(dst, (x, y, w, h), term_crit)
    
    
    
        x,y,w,h = track_window
    
        img2 = cv.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv.imshow('Seguimiento',img2)

        k = cv.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv.imwrite(chr(k)+".jpg",img2)
    else:
        break
    
    
cv.destroyAllWindows()
cap.release()

## Probando algunos otros trackers

[Link a la pagina donde encontre este snip](https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/)

In [5]:
(major_ver, minor_ver, subminor_ver) = (cv.__version__).split('.')

In [8]:
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[0]

if int(minor_ver) < 3:
    tracker = cv.Tracker_create(tracker_type)
else:
    if tracker_type == 'BOOSTING':
        tracker = cv.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv.TrackerMIL_create()
    if tracker_type == 'KCF':
        tracker = cv.TrackerKCF_create()
    if tracker_type == 'TLD':
        tracker = cv.TrackerTLD_create()
    if tracker_type == 'MEDIANFLOW':
        tracker = cv.TrackerMedianFlow_create()
    if tracker_type == 'GOTURN':
        tracker = cv.TrackerGOTURN_create()
    if tracker_type == 'MOSSE':
        tracker = cv.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv.TrackerCSRT_create()

# Read video
video = cv.VideoCapture(0)


# Read first frame.
ok, frame = video.read()


# Uncomment the line below to select a different bounding box
bbox = cv.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ok = tracker.init(frame, bbox)

while(1):
    # Read a new frame
    ok, frame = video.read()
    if not ok:
        break

    # Start timer
    timer = cv.getTickCount()

    # Update tracker
    ok, bbox = tracker.update(frame)

    # Calculate Frames per second (FPS)
    fps = cv.getTickFrequency() / (cv.getTickCount() - timer);

    # Draw bounding box
    if ok:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv.putText(frame, "Tracking failure detected", (100,80), cv.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

    # Display tracker type on frame
    cv.putText(frame, tracker_type + " Tracker", (100,20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

    # Display FPS on frame
    cv.putText(frame, "FPS : " + str(int(fps)), (100,50), cv.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

    # Display result
    cv.imshow("Tracking", frame)

    # Exit if ESC pressed
    k = cv.waitKey(1) & 0xff
    if k == 27 : break

cv.destroyAllWindows()
video.release()